# Base Notebook

Базовая ноутбука для создания задания

- [Пример установки дополнительных библиотек](#bullet-1)
- [Подключение линтера](#bullet-2)
- [Пример подключения к базе](#bullet-3)
- [1. Последняя запись ](#bullet-4)
- [2. Продажи и поставки](#bullet-5)
- [3. Товарооборот](#bullet-6)

## Пример установки дополнительных библиотек <a class="anchor" id="bullet-1"></a>

Просто `!pip install module-you-need`

In [ ]:
!pip install matplotlib

## Подключение линтера <a class="anchor" id="bullet-2"></a>

По приколу

In [ ]:
%load_ext pycodestyle_magic
%flake8_on

---

In [ ]:
import os

import numpy as np

import pandas as pd

import psycopg2

## Пример подключения к базе <a class="anchor" id="bullet-3"></a>

In [ ]:
POSTGRES = dict(
    host=os.environ['POSTGRES_HOST'],
    port=int(os.environ['POSTGRES_PORT']),
    dbname=os.environ['POSTGRES_DB'],
    user=os.environ['POSTGRES_USER'],
    password=os.environ['POSTGRES_PASSWORD']
)

In [ ]:
conn = psycopg2.connect(**POSTGRES)

try:
    df = pd.read_sql('select * from hello;', conn)
except Exception as ex:
    print(f"Something is wrong: {ex}")
finally:
    conn.close()

In [ ]:
df

## 1. Последняя запись <a class="anchor" id="bullet-4"></a>

Есть таблица `stocks`, где хранится запас товара в магазине в некоторый момент времени.

**product** | stock | **ts**
:-- | :-- | :--
Швабра | 15 | 2021-07-28 08:00:00
Швабра | 14 | 2021-07-28 09:15:00
Перфоратор | 2 | 2021-07-28 11:30:00
... | ... | ...
Швабра | 3 | 2021-07-29 19:13:00

**product** - наименование товара;  
**stock** - запас товара;  
**ts** - момент, в который запас принял текущее значение;

Первичный ключ: `(product, ts)`

### Задача

Методами языка SQL и(или) Python нужно найти последнее состояние запаса для каждого из товаров.

В результате выполнения запроса должен получиться датасет со следующими колонками:

- **product** - наименование товара;
- **stock** - последнее состояние запаса;
- **ts** - момент, в который запас принял указанное значение;

### Твой запрос

In [ ]:
query = """

"""

### Получение данных

In [ ]:
conn = psycopg2.connect(**POSTGRES)

try:
    df = pd.read_sql(query, conn)
except Exception as ex:
    print(f"Something is wrong: {ex}")
finally:
    conn.close()

In [ ]:
df

## 2. Продажи и поставки <a class="anchor" id="bullet-5"></a>

Добавим таблицу `products`, содержащую данные о стоимости товаров:

**product** | price
:-- | :--
Швабра | 1500
... | ...
Люстра | 2000

**product** - наименование товара, первичный ключ;  
**price** - цена продажи в рублях;

Будем рассматривать два основных типа изменения запаса товара:
- **Продажа** - запас уменьшился по сравнению с предыдущим значением, потому что-то купил товар
- **Поставка** - запас увеличился по сравнению с предыдущим значением

### Задача

Методами языка SQL и(или) Python на основе дынных таблицы `stocks` нужно найти суммарные дневные продажи и поставки за каждый день (в штуках и в деньгах).

В результате выполнения запроса должен получиться датасет со следующими колонками:

- **dt** - дата;
- **product** - наименование товара;
- **action_type** - тип изменения запаса: 'SALE', если продажа, 'DELIVERY', если поставка;
- **qty** - продажи товара в штуках в указанную дату;
- **turnover** - продажи товара в деньгах в указанную дату;

### Твой запрос

In [ ]:
query = """

"""

### Получение данных

In [ ]:
conn = psycopg2.connect(**POSTGRES)

try:
    df = pd.read_sql(query, conn)
except Exception as ex:
    print(f"Something is wrong: {ex}")
finally:
    conn.close()

In [ ]:
df

## 3 Товарооборот <a class="anchor" id="bullet-6"></a>

Добавим таблички `orders` и `orders_history` с данными заказов. 

Таблица `orders`:

**order_id** | product | qty | ts
:-- | :-- | :-- | :--
546593652094504602 | Газонокосилка | 9 | 2021-07-19 08:32:31
... | ... | ... | ...
1948869281921308769 | Лампочка | 4 | 2021-07-19 08:42:45

**order_id** - ID заказа, первичный ключ;  
**product** - наименование товара;  
**qty** - заказанное количество товара;  
**ts** - дата и время создания заказа;  

Таблица `orders_history`, отражает события, происходящие с заказом:

**order_id** |  status | **ts**
:-- | :-- | :--
546593652094504602 | CREATED | 2021-07-19 08:32:31
... | ... | ...
546593652094504602 | CANCELLED | 2021-07-19 09:14:53

**order_id** - ID заказа;  
**status** - статус заказа:

- 'CREATED' - заказ создан;
- 'CANCELLED' - заказ отменен;
- 'COMPLETED' - заказ исполнен;

**ts** - дата и время, когла заказ получил статус;

Первичный ключ: `(order_id, ts)`

Покупки могут быть двух типов:

- Оффлайн: клиенты покупают товар в магазине (в этом случае заказ не создается, а просто изменяется запас);
- Онлайн: клиенты покупают товара через сайт, при покупке создается заказ со статусом 'CREATED', запас товара уменьшается на заказанное количество не позже, чем через 30 секунд после создания заказа (из-за лага в интеграции систем). Если заказ по какой-либо причине отменяется, заказанное количество возвращается и запас товара увеличивается.

### Задача

Методами языка SQL и(или) Python получить суммарный дневной онлайн и оффлайн товарооборот (продано в рублях) каждого из товаров. Отмененные  незавершенные заказы не должны учитываться при расчете.

В результате выполнения запроса должен получиться датасет со следующими колонками:

- **dt** - дата;
- **product** - наименование товара;
- **channel** - канал продажи: 'ONLINE', если был заказ, 'OFFLINE', если товар куплен в магазине (заказа не было);
- **turnover** - товарооборот товара в указанную дату;

### Твой запрос

In [ ]:
query = """

"""

### Получение данных

In [ ]:
conn = psycopg2.connect(**POSTGRES)

try:
    df = pd.read_sql(query, conn)
except Exception as ex:
    print(f"Something is wrong: {ex}")
finally:
    conn.close()

In [ ]:
df